In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import us
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from pyseir.rt import infer_rt, constants

%matplotlib widget

In [ ]:
cd '~/PycharmProjects/covid-data-model'

In [ ]:
plt.ioff()

# To Look at AutoSigma

In [ ]:
fips = "33"  # New Hampshire

In [ ]:
input_df = infer_rt._generate_input_data(
    fips=fips,
    include_testing_correction=True,
    include_deaths=False,
    figure_collector=None,
)

In [ ]:
# Save a reference to instantiated engine (eventually I want to pull out the figure
# generation and saving so that I don't have to pass a display_name and fips into the class
engine = infer_rt.RtInferenceEngine(
    data=input_df,
    display_name=infer_rt._get_display_name(fips),
    fips=fips,
    include_deaths=False,
)

In [ ]:
dfs = []
for auto_sigma in [1, 3, 5, 10, 15, 20, 25, 30]:
    engine.max_scaling_sigma = auto_sigma
    tmp = engine.infer_all(plot=False)
    tmp['auto_sigma'] = auto_sigma
    dfs.append(tmp)
df = pd.concat(dfs)

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(nrows=2,
                       sharex=True,
                       figsize=(8,6))
sns.lineplot(x=df.index, y="Rt_MAP_composite", hue="auto_sigma", data=df, ax=ax[0], legend='full')
ax[0].set_title(f"Infer Rt with Varying AutoSigma for FIPS = {fips}")
ax[0].legend(loc="lower left", fontsize=8)
ax[0].axvline(pd.Timestamp.today() - pd.Timedelta(days=7), linestyle=":")

ax[1].plot(input_df.index, input_df['cases'])
ax[1].set(ylabel='New Cases')

fig.autofmt_xdate()
fig.show()